**First, get the data from the sqlite file

In [1]:
import sqlite3
import pandas as pd

db_location = "../Data/soccer/database.sqlite"

def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d


def get_table_as_pd_dataframe(table_name):
    query = 'SELECT * FROM ' + table_name + ';'

    con = sqlite3.connect(db_location)
    con.row_factory = dict_factory
    cur = con.cursor()

    cur.execute(query)
    table = cur.fetchall()

    con.close()

    return pd.DataFrame(table)


# ***********  get pandas DataFrames from sqlite file ************
countries = get_table_as_pd_dataframe('Country')
leagues = get_table_as_pd_dataframe('League')
matches = get_table_as_pd_dataframe('Match')
players_data = get_table_as_pd_dataframe('Player')
player_attributes = get_table_as_pd_dataframe('Player_Attributes')
teams_data = get_table_as_pd_dataframe('Team')
team_attributes = get_table_as_pd_dataframe('Team_Attributes')

**Drop useless columns

In [2]:
matches.drop(list(matches.filter(regex = '^[A-Z]+$')), axis = 1, inplace=True)
matches.drop(columns=['country_id', 'possession', 'corner', 'cross', 'card', 'foulcommit', 'shotoff', 'shoton', 'goal'], inplace=True)
players_data.drop(columns=['birthday', 'height', 'weight'], inplace=True)
player_attributes = player_attributes[['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating']]

**Afterwards merge some tables,

In [3]:
players = (players_data.merge(player_attributes, how='inner', on=['player_fifa_api_id', 'player_api_id'], suffixes=('', '_pa'), validate='one_to_many')).drop(columns=['id_pa', 'id'])
teams = (teams_data.merge(team_attributes, how='inner', on=['team_fifa_api_id', 'team_api_id'], suffixes=('', '_pa'), validate='one_to_many')).drop(columns=['id_pa', 'id'])

**and clean the data

In [4]:
teams['team_fifa_api_id'] = pd.to_numeric(teams['team_fifa_api_id'], downcast='integer', errors='coerce')

players = players.dropna()
players['overall_rating'] = players['overall_rating'].astype('int8')

countries = countries.dropna()
leagues = leagues.dropna()

# teams with buildUpPlayDribblingClass as 'little' will have buildUpPlayDribbling as NaN
teams.buildUpPlayDribbling = teams.buildUpPlayDribbling.fillna(20)
teams = teams.dropna()

# talvez dar drop de PSA PSD PSH pq e mts vezes NaN? depende da liga I guess
matches = matches.dropna(subset=['B365A', 'B365D', 'B365H', 'id', 'league_id', 'season', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal'])

**Some output

In [5]:
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
sb.set()

plt.figure(figsize=(25,15))
rating_series = players.overall_rating.value_counts()
sb.barplot(x=rating_series.index, y=rating_series)

In [6]:
filter = (players["player_api_id"] == 39902)
players.loc[filter, ["overall_rating"]].mean()

overall_rating    80.0
dtype: float64

In [7]:
pd.set_option('display.max_columns', None)
matches

id  league_id     season  stage                 date  match_api_id  \
0          1          1  2008/2009      1  2008-08-17 00:00:00        492473   
1          2          1  2008/2009      1  2008-08-16 00:00:00        492474   
2          3          1  2008/2009      1  2008-08-16 00:00:00        492475   
3          4          1  2008/2009      1  2008-08-17 00:00:00        492476   
4          5          1  2008/2009      1  2008-08-16 00:00:00        492477   
...      ...        ...        ...    ...                  ...           ...   
24552  24553      21518  2015/2016      9  2015-10-25 00:00:00       2030167   
24553  24554      21518  2015/2016      9  2015-10-24 00:00:00       2030168   
24554  24555      21518  2015/2016      9  2015-10-26 00:00:00       2030169   
24555  24556      21518  2015/2016      9  2015-10-24 00:00:00       2030170   
24556  24557      21518  2015/2016      9  2015-10-23 00:00:00       2030171   

       home_team_api_id  away_team_api_id  home_team_goal  away_team_goal  \
0                  9987              9993               1               1   
1                 10000              9994               0               0   
2                  9984              8635               0               3   
3                  9991              9998               5               0   
4                  7947              9985               1               3   
...                 ...               ...             ...             ...   
24552              9906             10267               2               1   
24553              9864              9783               2               0   
24554              8315              9869               3               0   
24555              7878              8603               1               1   
24556              8370              8558               3               0   

       home_player_X1  home_player_X2  home_player_X3  home_player_X4  \
0                 NaN             NaN             NaN             NaN   
1                 NaN             NaN             NaN             NaN   
2                 NaN             NaN             NaN             NaN   
3                 NaN             NaN             NaN             NaN   
4                 NaN             NaN             NaN             NaN   
...               ...             ...             ...             ...   
24552             1.0             2.0             4.0             6.0   
24553             1.0             2.0             4.0             6.0   
24554             1.0             2.0             4.0             6.0   
24555             1.0             2.0             4.0             6.0   
24556             1.0             2.0             4.0             6.0   

       home_player_X5  home_player_X6  home_player_X7  home_player_X8  \
0                 NaN             NaN             NaN             NaN   
1                 NaN             NaN             NaN             NaN   
2                 NaN             NaN             NaN             NaN   
3                 NaN             NaN             NaN             NaN   
4                 NaN             NaN             NaN             NaN   
...               ...             ...             ...             ...   
24552             8.0             3.0             5.0             7.0   
24553             8.0             4.0             6.0             3.0   
24554             8.0             4.0             5.0             6.0   
24555             8.0             3.0             5.0             7.0   
24556             8.0             4.0             6.0             3.0   

       home_player_X9  home_player_X10  home_player_X11  away_player_X1  \
0                 NaN              NaN              NaN             NaN   
1                 NaN              NaN              NaN             NaN   
2                 NaN              NaN              NaN             NaN   
3                 NaN              NaN              NaN             Na

In [8]:
matches = matches.dropna()
matches = matches.sort_values(by=['season', 'date'])
matches

id  league_id     season  stage                 date  match_api_id  \
4769    4770       4769  2008/2009      1  2008-08-09 00:00:00        483130   
4770    4771       4769  2008/2009      1  2008-08-09 00:00:00        483131   
4773    4774       4769  2008/2009      1  2008-08-09 00:00:00        483134   
4774    4775       4769  2008/2009      1  2008-08-09 00:00:00        483135   
4776    4777       4769  2008/2009      1  2008-08-09 00:00:00        483137   
...      ...        ...        ...    ...                  ...           ...   
24490  24491      21518  2015/2016     38  2016-05-15 00:00:00       2030531   
24491  24492      21518  2015/2016     38  2016-05-15 00:00:00       2030532   
24494  24495      21518  2015/2016     38  2016-05-15 00:00:00       2030535   
24496  24497      21518  2015/2016     38  2016-05-15 00:00:00       2030537   
4701    4702       1729  2015/2016     38  2016-05-17 00:00:00       1987600   

       home_team_api_id  away_team_api_id  home_team_goal  away_team_goal  \
4769               9827              7819               2               1   
4770               9746              9831               1               0   
4773               9829              9847               1               0   
4774               8481              8639               0               0   
4776               9874              9855               1               2   
...                 ...               ...             ...             ...   
24490              8603              8305               2               1   
24491              8558              8372               4               2   
24494              9864              8306               4               1   
24496              8370              8581               3               1   
4701              10260              8678               3               1   

       home_player_X1  home_player_X2  home_player_X3  home_player_X4  \
4769              1.0             2.0             4.0             6.0   
4770              1.0             2.0             4.0             6.0   
4773              1.0             2.0             4.0             6.0   
4774              1.0             4.0             6.0             8.0   
4776              1.0             2.0             4.0             6.0   
...               ...             ...             ...             ...   
24490             1.0             4.0             6.0             8.0   
24491             1.0             2.0             4.0             6.0   
24494             1.0             2.0             4.0             6.0   
24496             1.0             2.0             4.0             6.0   
4701              1.0             2.0             4.0             6.0   

       home_player_X5  home_player_X6  home_player_X7  home_player_X8  \
4769              8.0             2.0             4.0             6.0   
4770              8.0             2.0             4.0             6.0   
4773              8.0             2.0             4.0             6.0   
4774              2.0             1.0             3.0             5.0   
4776              8.0             2.0             4.0             6.0   
...               ...             ...             ...             ...   
24490             2.0             2.0             4.0             6.0   
24491             8.0             5.0             2.0             4.0   
24494             8.0             2.0             4.0             6.0   
24496             8.0             4.0             6.0             3.0   
4701              8.0             5.0             2.0             4.0   

       home_player_X9  home_player_X10  home_player_X11  away_player_X1  \
4769              8.0              4.0              6.0             1.0   
4770              8.0              4.0              6.0             1.0   
4773              8.0              4.0              6.0             1.0   
4774              7.0              9.0              5.0             1.

In [9]:
season_filter = (matches["season"] == "2008/2009")
matches_08_09 = matches.loc[season_filter]

matches_08_09

id  league_id     season  stage                 date  match_api_id  \
4769    4770       4769  2008/2009      1  2008-08-09 00:00:00        483130   
4770    4771       4769  2008/2009      1  2008-08-09 00:00:00        483131   
4773    4774       4769  2008/2009      1  2008-08-09 00:00:00        483134   
4774    4775       4769  2008/2009      1  2008-08-09 00:00:00        483135   
4776    4777       4769  2008/2009      1  2008-08-09 00:00:00        483137   
...      ...        ...        ...    ...                  ...           ...   
10567  10568      10257  2008/2009     38  2009-05-31 00:00:00        539930   
10568  10569      10257  2008/2009     38  2009-05-31 00:00:00        539931   
10570  10571      10257  2008/2009     38  2009-05-31 00:00:00        539933   
10573  10574      10257  2008/2009     38  2009-05-31 00:00:00        539936   
21829  21830      21518  2008/2009     38  2009-05-31 00:00:00        530676   

       home_team_api_id  away_team_api_id  home_team_goal  away_team_goal  \
4769               9827              7819               2               1   
4770               9746              9831               1               0   
4773               9829              9847               1               0   
4774               8481              8639               0               0   
4776               9874              9855               1               2   
...                 ...               ...             ...             ...   
10567              8636              8524               4               3   
10568              8600              8529               6               2   
10570              8535              8564               0               2   
10573              9875              8533               3               0   
21829              8558              9864               3               0   

       home_player_X1  home_player_X2  home_player_X3  home_player_X4  \
4769              1.0             2.0             4.0             6.0   
4770              1.0             2.0             4.0             6.0   
4773              1.0             2.0             4.0             6.0   
4774              1.0             4.0             6.0             8.0   
4776              1.0             2.0             4.0             6.0   
...               ...             ...             ...             ...   
10567             1.0             2.0             4.0             6.0   
10568             1.0             2.0             4.0             6.0   
10570             1.0             2.0             4.0             6.0   
10573             1.0             3.0             4.0             7.0   
21829             1.0             2.0             4.0             6.0   

       home_player_X5  home_player_X6  home_player_X7  home_player_X8  \
4769              8.0             2.0             4.0             6.0   
4770              8.0             2.0             4.0             6.0   
4773              8.0             2.0             4.0             6.0   
4774              2.0             1.0             3.0             5.0   
4776              8.0             2.0             4.0             6.0   
...               ...             ...             ...             ...   
10567             8.0             3.0             5.0             7.0   
10568             8.0             3.0             5.0             7.0   
10570             8.0             4.0             6.0             3.0   
10573             1.0             3.0             5.0             7.0   
21829             8.0             4.0             6.0             3.0   

       home_player_X9  home_player_X10  home_player_X11  away_player_X1  \
4769              8.0              4.0              6.0             1.0   
4770              8.0              4.0              6.0             1.0   
4773              8.0              4.0              6.0             1.0   
4774              7.0              9.0              5.0             1.

In [10]:
first_date = matches_08_09.loc[:, "date"].min()
last_date = matches_08_09.loc[:, "date"].max()

date_filter = ((players["date"] >= first_date) & (players["date"] <= last_date))

players_08_09 = players.loc[date_filter]

players_08_09

player_api_id          player_name  player_fifa_api_id  \
35             155782      Aaron Cresswell              189615   
36             155782      Aaron Cresswell              189615   
62             162549          Aaron Doran              186170   
83              30572        Aaron Galindo              140161   
84              30572        Aaron Galindo              140161   
...               ...                  ...                 ...   
183745          36491            Zsolt Low              111191   
183752          35506  Zurab Khizanishvili               47058   
183753          35506  Zurab Khizanishvili               47058   
183762          39902   Zvjezdan Misimovic              102359   
183763          39902   Zvjezdan Misimovic              102359   

                       date  overall_rating  
35      2009-02-22 00:00:00              47  
36      2008-08-30 00:00:00              53  
62      2009-02-22 00:00:00              59  
83      2009-02-22 00:00:00              75  
84      2008-08-30 00:00:00              73  
...                     ...             ...  
183745  2008-08-30 00:00:00              67  
183752  2009-02-22 00:00:00              70  
183753  2008-08-30 00:00:00              72  
183762  2009-02-22 00:00:00              78  
183763  2008-08-30 00:00:00              77  

[8684 rows x 5 columns]

In [11]:
%%time

id_dictionary = {
    "id": [],
    "home_team_api_id": []
}

for index, row in matches_08_09.iterrows():
    match_id = row["id"]
    home_team_id = row["home_team_api_id"]
    
    id_dictionary["id"].append(match_id)
    id_dictionary["home_team_api_id"].append(home_team_id)
        
df = pd.DataFrame(data = id_dictionary)

df

CPU times: user 218 ms, sys: 42 µs, total: 218 ms
Wall time: 217 ms


id  home_team_api_id
0      4770              9827
1      4771              9746
2      4774              9829
3      4775              8481
4      4777              9874
...     ...               ...
1718  10568              8636
1719  10569              8600
1720  10571              8535
1721  10574              9875
1722  21830              8558

[1723 rows x 2 columns]

In [12]:
%%time

def get_player_rating(date, player, season_matches, season_players):
    player_filter = (season_players["player_api_id"] == player)
    date_filter = (season_players["date"] <= date)
    filt = (player_filter & date_filter)
    player_info = season_players.loc[filt]
    
    if (player_info.empty):
        player_filter = (players["player_api_id"] == player)
        date_filter = (players["date"] <= date)
        filt = (player_filter & date_filter)
        player_info = players.loc[filt]
    
    closest_date = player_info.loc[:, "date"].max()
    
    date_filter = (player_info["date"] == closest_date)
    rating = player_info.loc[date_filter, "overall_rating"].item()
    return rating
  
    
def get_team_rating(condition, index, season_matches, season_players):
    date = season_matches.loc[index, "date"]
    
    team_rating = 0
    
    for player_num in range(1, 12):
        column = condition + "_player_" + str(player_num)
        player = season_matches.loc[index, column]
        player_rating = get_player_rating(date, player, season_matches, season_players)
        team_rating = team_rating + player_rating
    
    team_rating = team_rating/11.0
    return team_rating


rating_dictionary = {
    "home_team_rating": []
}

for index, row in matches_08_09.iterrows():
    home_team_rating = get_team_rating("home", index, matches_08_09, players_08_09)
    
    rating_dictionary["home_team_rating"].append(home_team_rating)
        
rating_df = pd.DataFrame(data = rating_dictionary)
df = df.join(rating_df)

df

CPU times: user 51.3 s, sys: 3.73 ms, total: 51.3 s
Wall time: 51.3 s


id  home_team_api_id  home_team_rating
0      4770              9827         74.454545
1      4771              9746         64.909091
2      4774              9829         67.909091
3      4775              8481         69.818182
4      4777              9874         69.545455
...     ...               ...               ...
1718  10568              8636         83.636364
1719  10569              8600         77.181818
1720  10571              8535         79.727273
1721  10574              9875         74.818182
1722  21830              8558         75.818182

[1723 rows x 3 columns]

In [13]:
%%time

def get_result(team_id, team_filter, date, season_matches):
    date_filter = (season_matches["date"] == date)
    filt = (team_filter & date_filter)
    match = season_matches.loc[filt]
    
    home_goals = match["home_team_goal"].item()
    away_goals = match["away_team_goal"].item()
    
    if match["home_team_api_id"].item() == team_id:
        if home_goals > away_goals:
            return 3
        elif home_goals < away_goals:
            return 0
        else:
            return 1
    else:
        if away_goals > home_goals:
            return 3
        elif away_goals < home_goals:
            return 0
        else:
            return 1

def get_form(team_id, team_filter, date, season_matches):
    date_filter = (season_matches["date"] < date)
    filt = (team_filter & date_filter)
    last_matches = season_matches.loc[filt]
    
    last_5_matches = last_matches.tail(5)
    
    if (last_5_matches.empty):
        return 0
    
    form = 0
    
    for index, row in last_5_matches.iterrows():
        match_date = row["date"]
        result = get_result(team_id, team_filter, match_date, season_matches)
        form = form + result
    
    index = last_5_matches.index
    num_matches = len(index)
    form = form / num_matches
    
    return form

form_dictionary = {
    "home_team_form": []
}

for index, row in matches_08_09.iterrows():
    team_id = row["home_team_api_id"]
    date = row["date"]
    team_filter = ((matches_08_09["home_team_api_id"] == team_id) | (matches_08_09["away_team_api_id"] == team_id))
    home_team_form = get_form(team_id, team_filter, date, matches_08_09)
    
    form_dictionary["home_team_form"].append(home_team_form)
        
form_df = pd.DataFrame(data = form_dictionary)
df = df.join(form_df)

df

CPU times: user 10.1 s, sys: 3.79 ms, total: 10.1 s
Wall time: 10.1 s


id  home_team_api_id  home_team_rating  home_team_form
0      4770              9827         74.454545             0.0
1      4771              9746         64.909091             0.0
2      4774              9829         67.909091             0.0
3      4775              8481         69.818182             0.0
4      4777              9874         69.545455             0.0
...     ...               ...               ...             ...
1718  10568              8636         83.636364             1.2
1719  10569              8600         77.181818             2.6
1720  10571              8535         79.727273             2.0
1721  10574              9875         74.818182             0.8
1722  21830              8558         75.818182             2.4

[1723 rows x 4 columns]

In [14]:
%%time

home_form_dictionary = {
    "home_team_form_at_home": []
}

for index, row in matches_08_09.iterrows():
    team_id = row["home_team_api_id"]
    date = row["date"]
    team_filter = (matches_08_09["home_team_api_id"] == team_id)
    home_team_form_at_home = get_form(team_id, team_filter, date, matches_08_09)
    
    home_form_dictionary["home_team_form_at_home"].append(home_team_form_at_home)
        
home_form_df = pd.DataFrame(data = home_form_dictionary)
df = df.join(home_form_df)

df

CPU times: user 8.8 s, sys: 12.1 ms, total: 8.82 s
Wall time: 8.79 s


id  home_team_api_id  home_team_rating  home_team_form  \
0      4770              9827         74.454545             0.0   
1      4771              9746         64.909091             0.0   
2      4774              9829         67.909091             0.0   
3      4775              8481         69.818182             0.0   
4      4777              9874         69.545455             0.0   
...     ...               ...               ...             ...   
1718  10568              8636         83.636364             1.2   
1719  10569              8600         77.181818             2.6   
1720  10571              8535         79.727273             2.0   
1721  10574              9875         74.818182             0.8   
1722  21830              8558         75.818182             2.4   

      home_team_form_at_home  
0                        0.0  
1                        0.0  
2                        0.0  
3                        0.0  
4                        0.0  
...                      ...  
1718                     2.2  
1719                     2.0  
1720                     2.4  
1721                     1.0  
1722                     2.0  

[1723 rows x 5 columns]

In [15]:
%%time

away_dictionary = {
    "away_team_api_id": [],
    "away_team_rating": [],
    "away_team_form": [],
    "away_team_form_away": []
}

for index, row in matches_08_09.iterrows():
    away_team_id = row["away_team_api_id"]
    
    away_team_rating = get_team_rating("away", index, matches_08_09, players_08_09)
    
    date = row["date"]
    
    team_filter = ((matches_08_09["home_team_api_id"] == away_team_id) | (matches_08_09["away_team_api_id"] == away_team_id))
    away_team_form = get_form(away_team_id, team_filter, date, matches_08_09)
    
    team_filter = (matches_08_09["away_team_api_id"] == away_team_id)
    away_team_form_away = get_form(away_team_id, team_filter, date, matches_08_09)
    
    away_dictionary["away_team_api_id"].append(away_team_id)
    away_dictionary["away_team_rating"].append(away_team_rating)
    away_dictionary["away_team_form"].append(away_team_form)
    away_dictionary["away_team_form_away"].append(away_team_form_away)
    
away_df = pd.DataFrame(data = away_dictionary)
df = df.join(away_df)

df

CPU times: user 1min 5s, sys: 6.53 ms, total: 1min 5s
Wall time: 1min 5s


id  home_team_api_id  home_team_rating  home_team_form  \
0      4770              9827         74.454545             0.0   
1      4771              9746         64.909091             0.0   
2      4774              9829         67.909091             0.0   
3      4775              8481         69.818182             0.0   
4      4777              9874         69.545455             0.0   
...     ...               ...               ...             ...   
1718  10568              8636         83.636364             1.2   
1719  10569              8600         77.181818             2.6   
1720  10571              8535         79.727273             2.0   
1721  10574              9875         74.818182             0.8   
1722  21830              8558         75.818182             2.4   

      home_team_form_at_home  away_team_api_id  away_team_rating  \
0                        0.0              7819         69.454545   
1                        0.0              9831         68.818182   
2                        0.0              9847         73.363636   
3                        0.0              8639         72.454545   
4                        0.0              9855         65.090909   
...                      ...               ...               ...   
1718                     2.2              8524         73.000000   
1719                     2.0              8529         72.090909   
1720                     2.4              8564         80.909091   
1721                     1.0              8533         71.000000   
1722                     2.0              9864         68.454545   

      away_team_form  away_team_form_away  
0                0.0                  0.0  
1                0.0                  0.0  
2                0.0                  0.0  
3                0.0                  0.0  
4                0.0                  0.0  
...              ...                  ...  
1718             1.0                  1.2  
1719             1.0                  1.0  
1720             1.4                  2.0  
1721             0.8                  1.8  
1722             2.2                  2.0  

[1723 rows x 9 columns]

In [16]:
%%time

def get_history_result(home_team_id, team_filter, date):
    date_filter = (matches["date"] == date)
    filt = (team_filter & date_filter)
    match = matches.loc[filt]
    
    home_goals = match["home_team_goal"].item()
    away_goals = match["away_team_goal"].item()
    
    if match["home_team_api_id"].item() == home_team_id:
        if home_goals > away_goals:
            return 3
        elif home_goals < away_goals:
            return 0
        else:
            return 1.5
    else:
        if away_goals > home_goals:
            return 3
        elif away_goals < home_goals:
            return 0
        else:
            return 1.5


def get_history(home_team_id, away_team_id, date):
    team_fiter = (((matches["home_team_api_id"] == home_team_id) & (matches["away_team_api_id"] == away_team_id)) 
                     | ((matches["home_team_api_id"] == away_team_id) & (matches["away_team_api_id"] == home_team_id)))
    date_filter = (matches["date"] < date)
    filt = (team_filter & date_filter)
    last_matches = matches.loc[filt]
    
    last_5_matches = last_matches.tail(5)
    
    if (last_5_matches.empty):
        return 1.5
    
    history = 0
    
    for index, row in last_5_matches.iterrows():
        match_date = row["date"]
        result = get_history_result(home_team_id, team_filter, match_date)
        history = history + result
    
    index = last_5_matches.index
    num_matches = len(index)
    history = history / num_matches
    
    return history


history_dictionary = {
    "history": []
}

for index, row in matches_08_09.iterrows():
    home_team_id = row["home_team_api_id"]
    away_team_id = row["away_team_api_id"]
    date = row["date"]
    history = get_history(home_team_id, away_team_id, date)
    
    history_dictionary["history"].append(history)
        
history_df = pd.DataFrame(data = history_dictionary)
df = df.join(history_df)

df

CPU times: user 2min 39s, sys: 43 ms, total: 2min 39s
Wall time: 2min 40s


id  home_team_api_id  home_team_rating  home_team_form  \
0      4770              9827         74.454545             0.0   
1      4771              9746         64.909091             0.0   
2      4774              9829         67.909091             0.0   
3      4775              8481         69.818182             0.0   
4      4777              9874         69.545455             0.0   
...     ...               ...               ...             ...   
1718  10568              8636         83.636364             1.2   
1719  10569              8600         77.181818             2.6   
1720  10571              8535         79.727273             2.0   
1721  10574              9875         74.818182             0.8   
1722  21830              8558         75.818182             2.4   

      home_team_form_at_home  away_team_api_id  away_team_rating  \
0                        0.0              7819         69.454545   
1                        0.0              9831         68.818182   
2                        0.0              9847         73.363636   
3                        0.0              8639         72.454545   
4                        0.0              9855         65.090909   
...                      ...               ...               ...   
1718                     2.2              8524         73.000000   
1719                     2.0              8529         72.090909   
1720                     2.4              8564         80.909091   
1721                     1.0              8533         71.000000   
1722                     2.0              9864         68.454545   

      away_team_form  away_team_form_away  history  
0                0.0                  0.0      1.5  
1                0.0                  0.0      1.5  
2                0.0                  0.0      1.5  
3                0.0                  0.0      1.5  
4                0.0                  0.0      1.5  
...              ...                  ...      ...  
1718             1.0                  1.2      2.1  
1719             1.0                  1.0      2.1  
1720             1.4                  2.0      2.1  
1721             0.8                  1.8      2.1  
1722             2.2                  2.0      2.1  

[1723 rows x 10 columns]

In [17]:
%%time

def get_season(year):
    year2 = year + 1
    years = str(year)
    year2s = str(year2)
        
    season = years + "/" + year2s
    
    return season


dictionary = {
        "id": [],
        "home_team_api_id": [],
        "home_team_rating": [],
        "home_team_form": [],
        "home_team_form_at_home": [],
        "away_team_api_id": [],
        "away_team_rating": [],
        "away_team_form": [],
        "away_team_form_away": [],
        "B365H": [],
        "B365D": [],
        "B365A": []
    }

year = 2008

while 1:
    season = get_season(year)
    print(season)
    season_filter = (matches["season"] == season)
    season_matches = matches.loc[season_filter]
    
    if season_matches.empty:
        break;
    
    first_date = season_matches.loc[:, "date"].min()
    last_date = season_matches.loc[:, "date"].max()
    date_filter = ((players["date"] >= first_date) & (players["date"] <= last_date))
    season_players = players.loc[date_filter]
     
    for index, row in season_matches.iterrows():
        match_id = row["id"]
        
        home_team_id = row["home_team_api_id"]
        
        home_team_rating = get_team_rating("home", index, season_matches, season_players)
        
        date = row["date"]
        team_filter = ((season_matches["home_team_api_id"] == home_team_id) | (season_matches["away_team_api_id"] == home_team_id))
        home_team_form = get_form(home_team_id, team_filter, date, season_matches)
        
        team_filter = (season_matches["home_team_api_id"] == home_team_id)
        home_team_form_at_home = get_form(home_team_id, team_filter, date, season_matches)
        
        away_team_id = row["away_team_api_id"]
    
        away_team_rating = get_team_rating("away", index, season_matches, season_players)
        
        team_filter = ((season_matches["home_team_api_id"] == away_team_id) | (season_matches["away_team_api_id"] == away_team_id))
        away_team_form = get_form(away_team_id, team_filter, date, season_matches)
        
        team_filter = (season_matches["away_team_api_id"] == away_team_id)
        away_team_form_away = get_form(away_team_id, team_filter, date, season_matches)
        
        b365h = row["B365H"]
        b365d = row["B365D"]
        b365a = row["B365A"]
        
        dictionary["id"].append(match_id)
        dictionary["home_team_api_id"].append(home_team_id)
        dictionary["home_team_rating"].append(home_team_rating)
        dictionary["home_team_form"].append(home_team_form)
        dictionary["home_team_form_at_home"].append(home_team_form_at_home)
        dictionary["away_team_api_id"].append(away_team_id)
        dictionary["away_team_rating"].append(away_team_rating)
        dictionary["away_team_form"].append(away_team_form)
        dictionary["away_team_form_away"].append(away_team_form_away)
        dictionary["B365H"].append(b365h)
        dictionary["B365D"].append(b365d)
        dictionary["B365A"].append(b365a)
        
    year = year + 1
        
df = pd.DataFrame(data = dictionary)

df

2008/2009
2009/2010
2010/2011
2011/2012
2012/2013
2013/2014
2014/2015
2015/2016
2016/2017
CPU times: user 37min 5s, sys: 476 ms, total: 37min 5s
Wall time: 37min 5s


id  home_team_api_id  home_team_rating  home_team_form  \
0       4770              9827         74.454545             0.0   
1       4771              9746         64.909091             0.0   
2       4774              9829         67.909091             0.0   
3       4775              8481         69.818182             0.0   
4       4777              9874         69.545455             0.0   
...      ...               ...               ...             ...   
19680  24491              8603         75.727273             1.0   
19681  24492              8558         74.363636             0.8   
19682  24495              9864         75.545455             0.8   
19683  24497              8370         72.909091             0.8   
19684   4702             10260         78.363636             2.0   

       home_team_form_at_home  away_team_api_id  away_team_rating  \
0                         0.0              7819         69.454545   
1                         0.0              9831         68.818182   
2                         0.0              9847         73.363636   
3                         0.0              8639         72.454545   
4                         0.0              9855         65.090909   
...                       ...               ...               ...   
19680                     1.8              8305         74.636364   
19681                     2.0              8372         73.090909   
19682                     1.6              8306         71.363636   
19683                     1.4              8581         73.272727   
19684                     2.6              8678         72.000000   

       away_team_form  away_team_form_away  B365H  B365D  B365A  
0                 0.0                  0.0   1.57   3.60   6.50  
1                 0.0                  0.0   2.30   3.00   3.40  
2                 0.0                  0.0   2.40   3.10   3.10  
3                 0.0                  0.0   2.15   3.10   3.60  
4                 0.0                  0.0   1.91   3.20   4.33  
...               ...                  ...    ...    ...    ...  
19680             1.0                  0.6   5.00   3.80   1.70  
19681             1.0                  0.4   2.00   3.60   3.70  
19682             1.0                  1.8   1.80   3.75   4.50  
19683             1.4                  0.2   1.33   5.25   9.00  
19684             0.8                  1.4   1.67   4.20   5.25  

[19685 rows x 12 columns]

In [18]:
odds_df = matches.loc[:, ["B365H", "B365D", "B365A"]]

odds_df

B365H  B365D  B365A
4769    1.57   3.60   6.50
4770    2.30   3.00   3.40
4773    2.40   3.10   3.10
4774    2.15   3.10   3.60
4776    1.91   3.20   4.33
...      ...    ...    ...
24490   5.00   3.80   1.70
24491   2.00   3.60   3.70
24494   1.80   3.75   4.50
24496   1.33   5.25   9.00
4701    1.67   4.20   5.25

[19685 rows x 3 columns]

In [19]:
%%time

#split data into training and test

from sklearn.model_selection import train_test_split

test_df = df.drop(["home_team_api_id", "away_team_api_id"], axis=1)

matches_train, matches_test, odds_train, odds_test = train_test_split(test_df.drop(["id", "B365H", "B365D", "B365A"], axis=1), odds_df, test_size=0.20)

CPU times: user 121 ms, sys: 8.03 ms, total: 129 ms
Wall time: 767 ms


In [20]:
#import multiple output regressor
from sklearn.multioutput import MultiOutputRegressor

In [26]:
%%time

#decision tree

from sklearn.tree import DecisionTreeRegressor

decision_tree = MultiOutputRegressor(DecisionTreeRegressor(random_state = 0))

decision_tree.fit(matches_train, odds_train)

decision_tree_output = decision_tree.predict(matches_test)

print(decision_tree.score(matches_test, odds_test))

decision_tree_output

0.4627572882557102
CPU times: user 269 ms, sys: 0 ns, total: 269 ms
Wall time: 273 ms


array([[2.9 , 3.8 , 3.  ],
       [1.25, 3.2 , 9.5 ],
       [2.3 , 3.4 , 3.5 ],
       ...,
       [2.75, 2.9 , 3.75],
       [2.25, 3.25, 5.5 ],
       [2.25, 4.2 , 4.2 ]])

In [22]:
%%time

#neural network

from sklearn.neural_network import MLPRegressor

neural_net = MultiOutputRegressor(MLPRegressor(max_iter=1000, random_state=1))

neural_net.fit(matches_train, odds_train)

neural_net_output = neural_net.predict(matches_test)

print(neural_net.score(matches_test, odds_test))

neural_net_output

0.6908067351897271
CPU times: user 26.4 s, sys: 12 s, total: 38.4 s
Wall time: 9.76 s


array([[2.19254769, 3.18701195, 3.04173188],
       [1.6159963 , 4.25501217, 6.67627139],
       [1.81904959, 3.5386503 , 3.89948648],
       ...,
       [2.67567175, 3.11656129, 2.44353453],
       [2.55772087, 3.31808894, 2.94316398],
       [2.01990635, 3.57248195, 4.37031995]])

In [23]:
%%time

#k nearest neighbor

from sklearn.neighbors import KNeighborsRegressor

knn = MultiOutputRegressor(KNeighborsRegressor())

knn.fit(matches_train, odds_train)

knn_output = knn.predict(matches_test)

print(knn.score(matches_test, odds_test))

knn_output

0.7063523850212704
CPU times: user 498 ms, sys: 84.1 ms, total: 582 ms
Wall time: 359 ms


array([[2.4  , 3.42 , 2.95 ],
       [1.598, 4.38 , 7.   ],
       [1.952, 3.696, 4.35 ],
       ...,
       [2.966, 3.31 , 2.626],
       [2.316, 3.346, 3.602],
       [1.96 , 3.43 , 4.07 ]])

In [24]:
%%time

#support vector machines

from sklearn.svm import SVR

svm = MultiOutputRegressor(SVR())

svm.fit(matches_train, odds_train)

svm_output = svm.predict(matches_test)

print(svm.score(matches_test, odds_test))

svm_output

0.4324007443787338
CPU times: user 42.8 s, sys: 0 ns, total: 42.8 s
Wall time: 42.8 s


array([[2.41201144, 3.35226842, 3.15994385],
       [1.73330084, 3.85344493, 5.69431456],
       [2.07509404, 3.54654578, 4.26707507],
       ...,
       [2.78474217, 3.330254  , 2.6055416 ],
       [2.68210887, 3.28953299, 2.93013988],
       [2.13331197, 3.54172377, 4.36771016]])